In [1]:
!pip install kmodes

In [50]:
import pandas as pd
import numpy as np
import time

import seaborn as sns
import matplotlib.pyplot as plt

In [51]:
%%time
def data_loader():
    train = pd.read_csv('train.csv')
    test = pd.read_csv('test.csv')
    sub = pd.read_csv('SampleSubmission.csv')
    #policy = pd.read_csv('policies.csv')
    #data_dict = pd.read_csv('DataDictionary.csv')
    return train,test, sub,

Wall time: 0 ns


In [67]:
%%time
train,test, sub,  = data_loader()

Wall time: 144 ms


In [68]:
df = pd.concat([train, test]).reset_index(drop=True)
ntrain = train.shape[0]

In [69]:
from kmodes.kmodes import KModes
import matplotlib.pyplot as plt
%matplotlib inline

In [70]:
df.head()

,claim_id,policy_number,claim_number,sum_insured,product,agent,class_of_business,risk_type,loss_date,client_type,renewal_frequency,primary_cause,secondary_cause,branch,target
0,ID_JZ7QWRHH57RO,pol000009680,claim0011359,300000.0,prod00027,ag00068,cob00031,rt00006,2021-05-13,ct0003,rf0001,pc0007,sc00022,br00006,82150.00
1,ID_KQ5TJMVLJ4EP,pol000006869,claim0005272,16000.0,prod00005,ag00226,cob00010,rt00032,2018-05-13,ct0003,rf0001,pc0007,sc00023,br00002,3780.24
2,ID_NYHI7WJGGIAE,pol000006135,claim0004147,3000.0,prod00029,ag00037,cob00010,rt00026,2018-08-18,ct0003,rf0001,pc0007,sc00024,br00002,2196.50
3,ID_ROMMDCXYUXN5,pol000008991,claim0009962,6000.0,prod00005,ag00037,cob00009,rt00032,2018-12-20,ct0003,rf0001,pc0007,sc00021,br00001,105.67
4,ID_2OTD9NX8L73D,pol000007768,claim0007346,26850.0,prod00015,ag00226,cob00034,rt00023,2018-11-07,ct0001,rf0001,pc0007,sc00021,br00001,1605.64


In [71]:
df['risk_business'] = df['class_of_business'] + '_' + df['risk_type']
df['product_agent'] = df['agent'] + '_' + df['product']
df['renwal_business'] = df['renewal_frequency'] + '_' + df['class_of_business']
df['product_class'] = df['class_of_business'] + '_' + df['product']

#New Features
df['claim_policy'] = df['claim_number'] + '_' + df['policy_number']
df['business_agent'] = df['class_of_business'] + '_' + df['agent']
df['primary_risk'] = df['primary_cause'] + '_' + df['risk_type']
df['secondary_risk'] = df['secondary_cause'] + '_' + df['risk_type']

df['year'] = [2022 - int(i.split('-')[0])  for i in df.loss_date]
df['cause'] = df['secondary_cause'] + '_' + df['primary_cause']
df['product_class'] = df['class_of_business'] + '_' + df['product']

id_claim = [i.split('m')[-1] for i in df['claim_number']]
df['id_claim'] = id_claim
df['idclaim_freq'] = df['id_claim'].map(df['id_claim'].value_counts())
df = df.drop('id_claim', axis = 1)

df['loss_date'] = pd.to_datetime(df['loss_date'], format='%Y-%m-%d')

for attr in ['day', 'month', 'week', 'dayofweek', 'weekofyear', 'days_in_month', 'is_month_start', 'is_month_end', 'dayofyear']:
    df[attr] = getattr(df['loss_date'].dt, attr)
df['is_weekend'] = (df['dayofweek'] >= 5)*1
df['fortnight'] = df['day']%15
df['which_fortnight'] = df['day']//15

df['branch_product'] = pd.Series(df.branch + '_' + df['product'])

C:\Users\Hp\AppData\Local\Temp/ipykernel_3756/3915202171.py:24: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df[attr] = getattr(df['loss_date'].dt, attr)


In [72]:
data = df[['risk_type', 'product_class', 'cause']]

kmode = KModes(n_clusters=3, init = "random", n_init = 5, verbose=1)
clusters = kmode.fit_predict(data)
#clusters

data.insert(0, "Cluster", clusters, True)


pd.DataFrame(data)['Cluster'].value_counts()

df[data.columns.to_list()] = data

df = df.drop(['risk_type', 'product_class', 'cause'], axis = 1)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 2690, cost: 18350.0
Run 1, iteration: 2/100, moves: 15, cost: 18350.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 71, cost: 18350.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 0, cost: 19532.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 1, cost: 22422.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 2255, cost: 19500.0
Run 5, iteration: 2/100, moves: 916, cost: 19500.0
Best run was number 1


df = df[['sum_insured', 'product', 'class_of_business', 'risk_type', 'client_type', 
    'renewal_frequency', 'primary_cause', 'secondary_cause', 'branch','renwal_business', 'product_class', 
         'cause', 'year', 'idclaim_freq','Cluster',
    'day', 'month', 'week', 'dayofweek', 'weekofyear', 'days_in_month', 
    'fortnight', 'which_fortnight', 'branch_product']]

In [73]:
df = df.drop('target', axis = 1)

In [74]:
df = df[['sum_insured', 'product', 'class_of_business', 'client_type', 
    'renewal_frequency', 'primary_cause', 'secondary_cause', 'branch','renwal_business',
         'year', 'idclaim_freq','Cluster',
    'day', 'month', 'week', 'dayofweek', 'weekofyear', 'days_in_month', 
    'fortnight', 'which_fortnight', 'branch_product','secondary_risk', 'primary_risk', 'claim_policy', 'business_agent']]

In [78]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder,OrdinalEncoder

In [76]:
features = [feat for feat in list(df) if feat != 'target' ]
cat_feat = np.where(df[features].dtypes != float)[0]

In [80]:
to_encode = [val for index, val in enumerate(features) if index in cat_feat]
df[to_encode] = df[to_encode].apply(LabelEncoder().fit_transform)

In [81]:
new_test = df[train.shape[0]:].reset_index(drop = True)
new_train = df[:train.shape[0]]

In [82]:
from sklearn import model_selection
from sklearn import metrics
from math import sqrt

In [93]:
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor

#lgbm_model = CatBoostRegressor(iterations=3000, random_state = 2000)


fold = model_selection.KFold(n_splits=10)

def metric(y_val, pred):
    return metrics.mean_absolute_error(y_val, pred)

In [94]:
lgbm_model = LGBMRegressor(n_estimators=1000)

In [95]:
%%time
def modelling_tree(algorithm,yy,X, test_df):
    cv_score=[]
    test_pred=[]
    train_score=[]
    xx = []
    for train_index, test_index in fold.split(X,pd.Series(yy)):
        
        X_trainx, X_val = X.iloc[train_index], X.iloc[test_index]
        y_trainx, y_val = pd.Series(yy).iloc[train_index], pd.Series(yy).iloc[test_index]

        algorithm.fit(X_trainx,y_trainx, eval_set=[(X_trainx,y_trainx),(X_val, y_val)])
        
        train_preds=algorithm.predict(X_trainx)
        
        train_score.append(metric(y_trainx, train_preds))
        
        val_preds=algorithm.predict(X_val)
        cv_score.append(metric(y_val, val_preds))

        test_p = algorithm.predict(test_df)
        test_pred.append(test_p)


    print(f'Training RMSE Score {np.mean(train_score)}')
    print(f'Validation RMSE Score {np.mean(cv_score)}')
    
    return test_pred

Wall time: 0 ns


In [96]:
%%time
test_pred = modelling_tree(algorithm = lgbm_model, yy = np.log1p(train.target), X = new_train, test_df = new_test)

[1]	training's l2: 11.724	valid_1's l2: 12.5764
[2]	training's l2: 11.1935	valid_1's l2: 12.1789
[3]	training's l2: 10.748	valid_1's l2: 11.8671
[4]	training's l2: 10.4169	valid_1's l2: 11.6281
[5]	training's l2: 10.1035	valid_1's l2: 11.4189
[6]	training's l2: 9.83145	valid_1's l2: 11.2274
[7]	training's l2: 9.59811	valid_1's l2: 11.0946
[8]	training's l2: 9.40626	valid_1's l2: 10.9487
[9]	training's l2: 9.24723	valid_1's l2: 10.8675
[10]	training's l2: 9.09167	valid_1's l2: 10.8073
[11]	training's l2: 8.96512	valid_1's l2: 10.7358
[12]	training's l2: 8.85292	valid_1's l2: 10.695
[13]	training's l2: 8.75273	valid_1's l2: 10.6435
[14]	training's l2: 8.66083	valid_1's l2: 10.6088
[15]	training's l2: 8.56939	valid_1's l2: 10.5689
[16]	training's l2: 8.48569	valid_1's l2: 10.5408
[17]	training's l2: 8.40623	valid_1's l2: 10.5214
[18]	training's l2: 8.33397	valid_1's l2: 10.5067
[19]	training's l2: 8.27052	valid_1's l2: 10.4907
[20]	training's l2: 8.20056	valid_1's l2: 10.4909
[21]	trainin

[229]	training's l2: 4.37218	valid_1's l2: 10.3978
[230]	training's l2: 4.36134	valid_1's l2: 10.4006
[231]	training's l2: 4.35419	valid_1's l2: 10.4076
[232]	training's l2: 4.34241	valid_1's l2: 10.4057
[233]	training's l2: 4.3314	valid_1's l2: 10.4114
[234]	training's l2: 4.32354	valid_1's l2: 10.4072
[235]	training's l2: 4.31461	valid_1's l2: 10.4055
[236]	training's l2: 4.30429	valid_1's l2: 10.4069
[237]	training's l2: 4.29774	valid_1's l2: 10.4101
[238]	training's l2: 4.28849	valid_1's l2: 10.4003
[239]	training's l2: 4.27894	valid_1's l2: 10.3986
[240]	training's l2: 4.26809	valid_1's l2: 10.3904
[241]	training's l2: 4.26069	valid_1's l2: 10.3861
[242]	training's l2: 4.25283	valid_1's l2: 10.3888
[243]	training's l2: 4.24328	valid_1's l2: 10.3887
[244]	training's l2: 4.23426	valid_1's l2: 10.383
[245]	training's l2: 4.22778	valid_1's l2: 10.3783
[246]	training's l2: 4.21854	valid_1's l2: 10.3812
[247]	training's l2: 4.214	valid_1's l2: 10.3779
[248]	training's l2: 4.20457	valid_

[467]	training's l2: 2.79204	valid_1's l2: 10.5531
[468]	training's l2: 2.78724	valid_1's l2: 10.5487
[469]	training's l2: 2.78426	valid_1's l2: 10.5492
[470]	training's l2: 2.77892	valid_1's l2: 10.5474
[471]	training's l2: 2.77295	valid_1's l2: 10.5524
[472]	training's l2: 2.76871	valid_1's l2: 10.5522
[473]	training's l2: 2.76517	valid_1's l2: 10.558
[474]	training's l2: 2.76072	valid_1's l2: 10.5574
[475]	training's l2: 2.75761	valid_1's l2: 10.5606
[476]	training's l2: 2.75262	valid_1's l2: 10.5615
[477]	training's l2: 2.74782	valid_1's l2: 10.5637
[478]	training's l2: 2.74269	valid_1's l2: 10.5709
[479]	training's l2: 2.73927	valid_1's l2: 10.574
[480]	training's l2: 2.73341	valid_1's l2: 10.5805
[481]	training's l2: 2.73033	valid_1's l2: 10.5784
[482]	training's l2: 2.72605	valid_1's l2: 10.5771
[483]	training's l2: 2.72021	valid_1's l2: 10.5737
[484]	training's l2: 2.71531	valid_1's l2: 10.5675
[485]	training's l2: 2.71143	valid_1's l2: 10.566
[486]	training's l2: 2.70838	valid

[693]	training's l2: 1.9852	valid_1's l2: 10.7387
[694]	training's l2: 1.98248	valid_1's l2: 10.7337
[695]	training's l2: 1.98104	valid_1's l2: 10.7327
[696]	training's l2: 1.97743	valid_1's l2: 10.7318
[697]	training's l2: 1.97455	valid_1's l2: 10.7347
[698]	training's l2: 1.97192	valid_1's l2: 10.7332
[699]	training's l2: 1.96901	valid_1's l2: 10.7362
[700]	training's l2: 1.96654	valid_1's l2: 10.7381
[701]	training's l2: 1.96395	valid_1's l2: 10.7376
[702]	training's l2: 1.96057	valid_1's l2: 10.7373
[703]	training's l2: 1.95735	valid_1's l2: 10.7419
[704]	training's l2: 1.95574	valid_1's l2: 10.7455
[705]	training's l2: 1.95333	valid_1's l2: 10.7467
[706]	training's l2: 1.95175	valid_1's l2: 10.7453
[707]	training's l2: 1.94982	valid_1's l2: 10.7494
[708]	training's l2: 1.94649	valid_1's l2: 10.7507
[709]	training's l2: 1.94368	valid_1's l2: 10.7496
[710]	training's l2: 1.94166	valid_1's l2: 10.7474
[711]	training's l2: 1.93803	valid_1's l2: 10.7431
[712]	training's l2: 1.93396	val

[928]	training's l2: 1.43484	valid_1's l2: 10.9155
[929]	training's l2: 1.43312	valid_1's l2: 10.9154
[930]	training's l2: 1.42984	valid_1's l2: 10.9176
[931]	training's l2: 1.42833	valid_1's l2: 10.9152
[932]	training's l2: 1.42588	valid_1's l2: 10.9152
[933]	training's l2: 1.42418	valid_1's l2: 10.9162
[934]	training's l2: 1.42233	valid_1's l2: 10.9161
[935]	training's l2: 1.42117	valid_1's l2: 10.9155
[936]	training's l2: 1.41983	valid_1's l2: 10.9166
[937]	training's l2: 1.41816	valid_1's l2: 10.9155
[938]	training's l2: 1.41698	valid_1's l2: 10.9147
[939]	training's l2: 1.4135	valid_1's l2: 10.917
[940]	training's l2: 1.41217	valid_1's l2: 10.918
[941]	training's l2: 1.40854	valid_1's l2: 10.9236
[942]	training's l2: 1.40764	valid_1's l2: 10.924
[943]	training's l2: 1.4055	valid_1's l2: 10.9224
[944]	training's l2: 1.40367	valid_1's l2: 10.9218
[945]	training's l2: 1.4021	valid_1's l2: 10.9243
[946]	training's l2: 1.40072	valid_1's l2: 10.9262
[947]	training's l2: 1.39809	valid_1'

...

[74]	training's l2: 6.54617	valid_1's l2: 9.4623
[75]	training's l2: 6.52395	valid_1's l2: 9.46991
[76]	training's l2: 6.5007	valid_1's l2: 9.46237
[77]	training's l2: 6.47626	valid_1's l2: 9.47899
[78]	training's l2: 6.45355	valid_1's l2: 9.46413
[79]	training's l2: 6.43931	valid_1's l2: 9.46084
[80]	training's l2: 6.41827	valid_1's l2: 9.44581
[81]	training's l2: 6.39774	valid_1's l2: 9.45324
[82]	training's l2: 6.38161	valid_1's l2: 9.45484
[83]	training's l2: 6.36279	valid_1's l2: 9.45891
[84]	training's l2: 6.34481	valid_1's l2: 9.45961
[85]	training's l2: 6.33077	valid_1's l2: 9.45751
[86]	training's l2: 6.31253	valid_1's l2: 9.45764
[87]	training's l2: 6.28897	valid_1's l2: 9.45808
[88]	training's l2: 6.27296	valid_1's l2: 9.45788
[89]	training's l2: 6.25072	valid_1's l2: 9.44761
[90]	training's l2: 6.23137	valid_1's l2: 9.44384
[91]	training's l2: 6.21637	valid_1's l2: 9.43354
[92]	training's l2: 6.1987	valid_1's l2: 9.43083
[93]	training's l2: 6.17368	valid_1's l2: 9.43477
[94

[317]	training's l2: 3.64812	valid_1's l2: 9.65787
[318]	training's l2: 3.64064	valid_1's l2: 9.65788
[319]	training's l2: 3.63257	valid_1's l2: 9.66713
[320]	training's l2: 3.62604	valid_1's l2: 9.66317
[321]	training's l2: 3.61989	valid_1's l2: 9.66443
[322]	training's l2: 3.61416	valid_1's l2: 9.67321
[323]	training's l2: 3.60733	valid_1's l2: 9.67282
[324]	training's l2: 3.59889	valid_1's l2: 9.67472
[325]	training's l2: 3.59412	valid_1's l2: 9.67771
[326]	training's l2: 3.58829	valid_1's l2: 9.67644
[327]	training's l2: 3.57975	valid_1's l2: 9.67704
[328]	training's l2: 3.57463	valid_1's l2: 9.68154
[329]	training's l2: 3.56843	valid_1's l2: 9.68042
[330]	training's l2: 3.56245	valid_1's l2: 9.68118
[331]	training's l2: 3.55603	valid_1's l2: 9.68017
[332]	training's l2: 3.55	valid_1's l2: 9.67521
[333]	training's l2: 3.54101	valid_1's l2: 9.6799
[334]	training's l2: 3.53592	valid_1's l2: 9.6763
[335]	training's l2: 3.52891	valid_1's l2: 9.67259
[336]	training's l2: 3.52295	valid_1

[565]	training's l2: 2.4019	valid_1's l2: 9.96384
[566]	training's l2: 2.39941	valid_1's l2: 9.96436
[567]	training's l2: 2.39562	valid_1's l2: 9.9685
[568]	training's l2: 2.39212	valid_1's l2: 9.96877
[569]	training's l2: 2.38887	valid_1's l2: 9.97579
[570]	training's l2: 2.38486	valid_1's l2: 9.97885
[571]	training's l2: 2.37946	valid_1's l2: 9.98401
[572]	training's l2: 2.37669	valid_1's l2: 9.98709
[573]	training's l2: 2.37295	valid_1's l2: 9.99182
[574]	training's l2: 2.37044	valid_1's l2: 9.99026
[575]	training's l2: 2.36814	valid_1's l2: 9.98928
[576]	training's l2: 2.36508	valid_1's l2: 9.99242
[577]	training's l2: 2.36137	valid_1's l2: 9.99378
[578]	training's l2: 2.35775	valid_1's l2: 9.99126
[579]	training's l2: 2.35307	valid_1's l2: 9.99387
[580]	training's l2: 2.34999	valid_1's l2: 9.99648
[581]	training's l2: 2.346	valid_1's l2: 9.99503
[582]	training's l2: 2.34238	valid_1's l2: 9.9975
[583]	training's l2: 2.3394	valid_1's l2: 9.99898
[584]	training's l2: 2.33499	valid_1'

[736]	training's l2: 1.87566	valid_1's l2: 10.1398
[737]	training's l2: 1.87327	valid_1's l2: 10.14
[738]	training's l2: 1.86937	valid_1's l2: 10.1432
[739]	training's l2: 1.86711	valid_1's l2: 10.1491
[740]	training's l2: 1.86518	valid_1's l2: 10.1514
[741]	training's l2: 1.86324	valid_1's l2: 10.1515
[742]	training's l2: 1.86032	valid_1's l2: 10.1503
[743]	training's l2: 1.85808	valid_1's l2: 10.1499
[744]	training's l2: 1.85491	valid_1's l2: 10.1447
[745]	training's l2: 1.85156	valid_1's l2: 10.1431
[746]	training's l2: 1.84873	valid_1's l2: 10.1386
[747]	training's l2: 1.8457	valid_1's l2: 10.1426
[748]	training's l2: 1.84277	valid_1's l2: 10.1401
[749]	training's l2: 1.8404	valid_1's l2: 10.1385
[750]	training's l2: 1.83887	valid_1's l2: 10.1424
[751]	training's l2: 1.83572	valid_1's l2: 10.1392
[752]	training's l2: 1.83423	valid_1's l2: 10.1413
[753]	training's l2: 1.83262	valid_1's l2: 10.1408
[754]	training's l2: 1.83104	valid_1's l2: 10.1419
[755]	training's l2: 1.82953	valid_

[901]	training's l2: 1.50818	valid_1's l2: 10.2846
[902]	training's l2: 1.50579	valid_1's l2: 10.2846
[903]	training's l2: 1.50366	valid_1's l2: 10.2865
[904]	training's l2: 1.50122	valid_1's l2: 10.2875
[905]	training's l2: 1.49874	valid_1's l2: 10.2864
[906]	training's l2: 1.49601	valid_1's l2: 10.2862
[907]	training's l2: 1.49347	valid_1's l2: 10.2882
[908]	training's l2: 1.49214	valid_1's l2: 10.2899
[909]	training's l2: 1.49014	valid_1's l2: 10.2917
[910]	training's l2: 1.48757	valid_1's l2: 10.2901
[911]	training's l2: 1.4862	valid_1's l2: 10.2933
[912]	training's l2: 1.48446	valid_1's l2: 10.2918
[913]	training's l2: 1.48315	valid_1's l2: 10.2939
[914]	training's l2: 1.48181	valid_1's l2: 10.2929
[915]	training's l2: 1.47944	valid_1's l2: 10.2939
[916]	training's l2: 1.47819	valid_1's l2: 10.2977
[917]	training's l2: 1.47599	valid_1's l2: 10.2995
[918]	training's l2: 1.47422	valid_1's l2: 10.2978
[919]	training's l2: 1.4725	valid_1's l2: 10.3009
[920]	training's l2: 1.47028	vali

In [ ]:
Shrink model to first 677 iterations.
Training RMSE Score 1.7742831513487345
Validation RMSE Score 2.0662827964790167
Wall time: 2min 27s

In [ ]:
Training RMSE Score 1.7708831781810859
Validation RMSE Score 2.068340989095058
Wall time: 1min 20s

In [49]:
cat_sub = np.mean(np.exp(test_pred) +1, axis = 0)
sub['target'] = cat_sub
sub['target'].describe()
sub.to_csv('1st_model.csv', index = False)